In [1]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import string as s
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import os
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to C:\Users\BINJAL
[nltk_data]     SONI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\BINJAL
[nltk_data]     SONI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\BINJAL
[nltk_data]     SONI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv(r'C:\Users\BINJAL SONI\OneDrive\Desktop\PDPU-Master\2nd Sem\Capstone Project\Fake News Detection\Datasets\21-02-2023\Constraint_Train.csv')
df.head(5)

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [3]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


In [3]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
le = LabelEncoder()



In [4]:
le = LabelEncoder()

# Fit and transform the 'color' column
df['label'] = le.fit_transform(df['label'])

print(df)

        id                                              tweet  label
0        1  The CDC currently reports 99031 deaths. In gen...      1
1        2  States reported 1121 deaths a small rise from ...      1
2        3  Politically Correct Woman (Almost) Uses Pandem...      0
3        4  #IndiaFightsCorona: We have 1524 #COVID testin...      1
4        5  Populous states can generate large case counts...      1
...    ...                                                ...    ...
6415  6416  A tiger tested positive for COVID-19 please st...      0
6416  6417  ???Autopsies prove that COVID-19 is??� a blood...      0
6417  6418  _A post claims a COVID-19 vaccine has already ...      0
6418  6419  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund      0
6419  6420  It has been 93 days since the last case of COV...      1

[6420 rows x 3 columns]


In [5]:

print(df)

        id                                              tweet  label
0        1  The CDC currently reports 99031 deaths. In gen...      1
1        2  States reported 1121 deaths a small rise from ...      1
2        3  Politically Correct Woman (Almost) Uses Pandem...      0
3        4  #IndiaFightsCorona: We have 1524 #COVID testin...      1
4        5  Populous states can generate large case counts...      1
...    ...                                                ...    ...
6415  6416  A tiger tested positive for COVID-19 please st...      0
6416  6417  ???Autopsies prove that COVID-19 is??� a blood...      0
6417  6418  _A post claims a COVID-19 vaccine has already ...      0
6418  6419  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund      0
6419  6420  It has been 93 days since the last case of COV...      1

[6420 rows x 3 columns]


In [6]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,0
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,0
6417,6418,_A post claims a COVID-19 vaccine has already ...,0
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,0


In [7]:
df = df.set_index('id')

In [8]:
df

,tweet,label
id,,
1,The CDC currently reports 99031 deaths. In gen...,1
2,States reported 1121 deaths a small rise from ...,1
3,Politically Correct Woman (Almost) Uses Pandem...,0
4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
5,Populous states can generate large case counts...,1
...,...,...
6416,A tiger tested positive for COVID-19 please st...,0
6417,???Autopsies prove that COVID-19 is??� a blood...,0
6418,_A post claims a COVID-19 vaccine has already ...,0


In [9]:
df = df.dropna()

In [10]:
X = df.drop('label', axis = 1)

In [11]:
y = df['label']

In [12]:
X.shape

(6420, 1)

In [13]:
y.shape

(6420,)

In [14]:
voc_size = 5000

msg = X.copy()

In [15]:
#reset the index to get the proper count
msg.reset_index(inplace = True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to C:\Users\BINJAL
[nltk_data]     SONI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\BINJAL
[nltk_data]     SONI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def tokenization(msg):
    lst=text.split()
    return lst
exx2=exx.apply(tokenization)

def lowercasing(lst):
    new_lst=[]
    for i in lst:
        i=i.lower()
        new_lst.append(i)
    return new_lst
exx2=exx2.apply(lowercasing)

def remove_punctuations(lst):
    new_lst=[]
    for i in lst:
        for j in s.punctuation:
            i=i.replace(j,'')
        new_lst.append(i)
    return new_lst
exx2=exx2.apply(remove_punctuations)
#test_x=test_x.apply(remove_punctuations)

def remove_numbers(lst):
    nodig_lst=[]
    new_lst=[]
    for i in lst:
        for j in s.digits:    
            i=i.replace(j,'')
        nodig_lst.append(i)
    for i in nodig_lst:
        if i!='':
            new_lst.append(i)
    return new_lst
exx2=exx2.apply(remove_numbers)
#test_x=test_x.apply(remove_numbers)

def remove_stopwords(lst):
    stop=stopwords.words('english')
    new_lst=[]
    for i in lst:
        if i not in stop:
            new_lst.append(i)
    return new_lst

exx2=exx2.apply(remove_stopwords)
#test_x=test_x.apply(remove_stopwords)

def remove_spaces(lst):
    new_lst=[]
    for i in lst:
        i=i.strip()
        new_lst.append(i)
    return new_lst
exx2=exx2.apply(remove_spaces)
#test_x=test_x.apply(remove_spaces)

from sklearn.feature_extraction.text import TfidfVectorizer
def TfidfVectorizer(lst):
    new_lst=[]
    for i in lst:
        i=TfidfVectorizer.TfidfVectorizer(i)
        new_lst.append(i)
    return new_lst
exx2=exx2.apply(TfidfVectorizer)
#test_x=test_x.apply(TfidfVectorizer)

corpus = []
for i in range(0, len(msg)):
   
    review = re.sub('[^a-zA-Z]', ' ', msg['tweet'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
from tensorflow.keras.layers import Bidirectional

In [19]:
len(corpus)

6420

In [20]:
msg['tweet'][0]

'The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.'

In [21]:
onehot_representation = [one_hot(words, voc_size) for words in corpus]

In [22]:
sent_length = 20
embeded_docs = pad_sequences(onehot_representation, padding = "pre", maxlen = sent_length)

embeded_docs[0]

array([   0,    0, 1338, 3719, 3629,  187, 1862,  518,  187, 3720, 2734,
        867, 2514, 4936,  187, 2468,  366, 4004,  120, 4998])

In [23]:
X_final = np.array(embeded_docs)

In [24]:
y_final = np.array(y)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [26]:
from keras.utils import to_categorical

# Build the LSTM model
dimension = 40
lstm_model = Sequential()
lstm_model.add(Embedding(voc_size, dimension, input_length = sent_length))
lstm_model.add(Bidirectional(LSTM(100)))
lstm_model.add(Dense(1, activation = "sigmoid"))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#lstm_model.fit(X_train, to_categorical(y_train), epochs=10, batch_size=32, validation_data=(X_test, to_categorical(y_test)))

In [27]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [28]:
nb = GaussianNB()
svm_classifier = svm.SVC(kernel='linear')

In [29]:
# Voting Classifier
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('lstm', lstm_model), ('svm', svm_classifier), ('nb', nb)], voting='soft')

In [30]:
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)
svm_classifier.fit(X_train.reshape(X_train.shape[0], -1), y_train)
nb.fit(X_train.reshape(X_train.shape[0], -1), y_train)


Epoch 1/10
161/161 [==============================] - 7s 15ms/step - loss: 0.3354 - accuracy: 0.8464
Epoch 2/10
161/161 [==============================] - 5s 29ms/step - loss: 0.1200 - accuracy: 0.9550
Epoch 3/10
161/161 [==============================] - 3s 17ms/step - loss: 0.0442 - accuracy: 0.9858
Epoch 4/10
161/161 [==============================] - 5s 29ms/step - loss: 0.0230 - accuracy: 0.9932
Epoch 5/10
161/161 [==============================] - 3s 17ms/step - loss: 0.0119 - accuracy: 0.9973
Epoch 6/10
161/161 [==============================] - 5s 29ms/step - loss: 0.0073 - accuracy: 0.9981
Epoch 7/10
161/161 [==============================] - 3s 20ms/step - loss: 0.0076 - accuracy: 0.9988
Epoch 8/10
161/161 [==============================] - 4s 28ms/step - loss: 0.0033 - accuracy: 0.9992
Epoch 9/10
161/161 [==============================] - 4s 24ms/step - loss: 9.5531e-04 - accuracy: 0.9998
Epoch 10/10
161/161 [==============================] - 4s 23ms/step - loss: 0.0279 - ac

GaussianNB()

In [40]:
voting_clf = VotingClassifier(estimators=[('nb', nb), ('svm_classifier', svm_classifier), ('LSTM', clf_model)], voting='hard')

In [41]:
# Convert the sequential model to a classifier
clf_model = Sequential()
clf_model.add(lstm_model)
clf_model.add(Dense(1, activation='sigmoid'))

In [42]:
voting_clf = VotingClassifier(estimators=[('nb', nb), ('svm_classifier', svm_classifier), ('LSTM', clf_model)], voting='hard')

In [44]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import VotingClassifier

# Fit the voting classifier
voting_clf.fit(X_train, y_train)

ValueError: The estimator Sequential should be a classifier.

In [38]:
y_pred = voting_clf.predict(X_test)
y_pred



AttributeError: 'VotingClassifier' object has no attribute 'estimators_'

In [ ]:
from sklearn.metrics import confusion_matrix
p_pred = voting_clf.predict(X_test)
p_pred = p_pred.flatten()
print(p_pred.round(2))
y_pred = np.where(p_pred > 0.5, 1, 0)
print(y_pred)
cf_matrix=confusion_matrix(y_test, y_pred)
print(cf_matrix)



In [ ]:
import seaborn as sns
ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True,fmt='.2%', cmap='Blues')
ax.set_title('Confusion Matrix with labels and percentage\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])
plt.show()



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))



In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))



In [ ]:
import joblib

joblib.dump(voting_clf, 'hybrid fake_model for lstm, svm, nb without clickbait.joblib')

In [ ]:
#Training a model

from tensorflow.keras.optimizers.legacy import Adam